In [1]:
import lightgbm
import pandas as pd
import numpy as np

In [2]:
DATA_PATH = r"../../../../../Data/Elo Merchant Category Recommendation/"

In [3]:
train1 = pd.read_csv(DATA_PATH + "train1.csv", header = "infer", sep = ",")

In [4]:
test1 = pd.read_csv(DATA_PATH + "test1.csv", header = "infer", sep = ",")

In [5]:
test2 = pd.read_csv(DATA_PATH + "test2.csv", header = "infer", sep = ",")

In [6]:
test3 = pd.read_csv(DATA_PATH + "test3.csv", header = "infer", sep = ",")

In [7]:
part = np.random.rand(len(train1))<0.7

In [8]:
valid = train1[~part]

In [9]:
train = train1[part]

In [10]:
train_card_id = train.loc[:,'card_id']
valid_card_id = valid.loc[:,'card_id']
test_card_id1 = test1.loc[:,'card_id']
test_card_id2 = test2.loc[:,'card_id']
test_card_id3 = test3.loc[:,'card_id']
x_train = train.loc[:,~train.columns.isin(['target','card_id'])]
x_valid = valid.loc[:,~valid.columns.isin(['target','card_id'])]
x_test1 = test1.loc[:,~test1.columns.isin(['card_id'])]
x_test2 = test2.loc[:,~test2.columns.isin(['card_id'])]
x_test3 = test3.loc[:,~test3.columns.isin(['card_id'])]
y_train = train.loc[:,'target']
y_valid = valid.loc[:,'target']

In [11]:
lgbm_train = lightgbm.Dataset(x_train,y_train)

In [12]:
lgbm_eval = lightgbm.Dataset(x_valid,y_valid)

In [13]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 2
}

In [14]:
gbm = lightgbm.train(params,
                lgbm_train,
                num_boost_round=20,
                valid_sets=lgbm_eval,
                early_stopping_rounds=5)

[1]	valid_0's l2: 14.9161	valid_0's l1: 1.59963
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 14.9129	valid_0's l1: 1.59966
[3]	valid_0's l2: 14.9066	valid_0's l1: 1.59946
[4]	valid_0's l2: 14.9039	valid_0's l1: 1.5996
[5]	valid_0's l2: 14.8996	valid_0's l1: 1.59961
[6]	valid_0's l2: 14.8948	valid_0's l1: 1.59965
[7]	valid_0's l2: 14.8908	valid_0's l1: 1.59982
[8]	valid_0's l2: 14.8882	valid_0's l1: 1.6
Early stopping, best iteration is:
[3]	valid_0's l2: 14.9066	valid_0's l1: 1.59946


In [15]:
y_pred1 = gbm.predict(x_test1,num_iteration=gbm.best_iteration)

In [16]:
y_pred2 = gbm.predict(x_test2,num_iteration=gbm.best_iteration)

In [17]:
y_pred3 = gbm.predict(x_test3,num_iteration=gbm.best_iteration)

In [18]:
result1 = pd.concat([test_card_id1,pd.Series(y_pred1)],axis = 1,keys = ["card_id","target"])

In [19]:
result2 = pd.concat([test_card_id2,pd.Series(y_pred2)],axis = 1,keys = ["card_id","target"])

In [20]:
result3 = pd.concat([test_card_id3,pd.Series(y_pred3)],axis = 1,keys = ["card_id","target"])

In [21]:
result1.to_csv(DATA_PATH + 'result1.csv', header='infer', sep=',', index=False) #3.929 place - 398

In [22]:
result2.to_csv(DATA_PATH + 'result2.csv', header='infer', sep=',', index=False) #3.929 place - 398

In [23]:
result3.to_csv(DATA_PATH + 'result3.csv', header='infer', sep=',', index=False) #3.929 place - 398